# Описание проекта

Нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. А так же проанализировать возможную прибыль и риски.

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# 1. Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np, scipy.stats as st
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from numpy.random import RandomState

In [2]:
for i in range(3): #выгрузка датасетов geo_data_0, geo_data_1, geo_data_2 и удаление колонок id
    path = '/datasets/geo_data_{}.csv'.format(i)
    df_name = 'data_{}'.format(i)
    globals()[df_name] = pd.read_csv(path).drop(['id'], axis=1)

In [3]:
def sampling(data):
    data_train, data_valid = train_test_split(data, test_size=0.25, random_state=12345)
    features_train = data_train.drop(['product'], axis=1)
    target_train = data_train['product']  
    features_valid = data_valid.drop(['product'], axis=1)
    target_valid = data_valid['product']
    return features_train, target_train, features_valid, target_valid

In [4]:
features_train_0, target_train_0, features_valid_0, target_valid_0 = sampling(data_0)

In [5]:
features_train_1, target_train_1, features_valid_1, target_valid_1 = sampling(data_1)

In [6]:
features_train_2, target_train_2, features_valid_2, target_valid_2 = sampling(data_2)

# 2. Обучение и проверка модели

In [7]:
def log_reg(features_train, features_valid, target_train, target_valid): #функция для тренировки 
#модели линейной регрессии и предсказания запасов сырья, а так же рачет метрик RMSE, R2, mae и вывод их на экран
    model = LinearRegression()
    model.fit(features_train,target_train)
    predicted_valid = model.predict(features_valid)
    predicted_mean = predicted_valid.mean()
    print('Средний запас сырья одного месторождения (тыс. баррелей): ', predicted_mean)
    rmse = mean_squared_error(target_valid, predicted_valid)**0.5
    print('RMSE модели =', rmse)
    r2 = r2_score(target_valid, predicted_valid)
    print('R2 =', r2)
    mae = mean_absolute_error(target_valid, predicted_valid)
    print('Среднее абсолютное отклонение =', mae)
    return predicted_valid, target_valid, predicted_mean, rmse, r2, mae

#### Первый регион.

In [8]:
predicted_valid_0, target_valid_0, predicted_mean_0, rmse_0, r2_0, mae_0 = log_reg(features_train_0, features_valid_0, target_train_0, target_valid_0)

Средний запас сырья одного месторождения (тыс. баррелей):  92.59256778438038
RMSE модели = 37.5794217150813
R2 = 0.27994321524487786
Среднее абсолютное отклонение = 30.919600777151313


#### Второй регион.

In [9]:
predicted_valid_1, target_valid_1, predicted_mean_1, rmse_1, r2_1, mae_1 = log_reg(features_train_1, features_valid_1, target_train_1, target_valid_1)

Средний запас сырья одного месторождения (тыс. баррелей):  68.728546895446
RMSE модели = 0.893099286775616
R2 = 0.9996233978805127
Среднее абсолютное отклонение = 0.718766244212475


#### Третий регион.

In [10]:
predicted_valid_2, target_valid_2, predicted_mean_2, rmse_2, r2_2, mae_2 = log_reg(features_train_2, features_valid_2, target_train_2, target_valid_2)

Средний запас сырья одного месторождения (тыс. баррелей):  94.96504596800489
RMSE модели = 40.02970873393434
R2 = 0.20524758386040443
Среднее абсолютное отклонение = 32.792652105481814


Модель построенная на данных второго региона наиболее одыкватная.

# 3. Подготовка к расчёту прибыли

In [11]:
SEL_BY_REGION = 500                # количество исследований на регион
FIELD_DEVELOP_BUDGET = 10000000000 # бюджет на разработку месторождений
COST_OF_ONE_WELL = 50000000        # стоимость бурения одной скважины
PROFIT_PER_BARREL = 4500000        # тысяча баррелей сырья приносит прибыли
RISK_OF_DAMAGES = 0.025            # не рассматривать регионы, в которых риск убытков выше 2.5%

1. Условие минимального среднего объема сырья - это нулевая операционная прибыль.
2. Следовательно валовая прибыль будет равна операционным расходам (FIELD_DEVELOP_BUDGET).
3. Валовая прибыль = оборот - себестоимость
4. Оборот(минимальный средний объем сырья * PROFIT_PER_BARREL) = FIELD_DEVELOP_BUDGET

In [12]:
max_number_of_deposits = FIELD_DEVELOP_BUDGET / COST_OF_ONE_WELL

In [13]:
max_number_of_deposits = int(max_number_of_deposits) #максимальное количесвто точек разработки на регион

In [14]:
max_number_of_deposits

200

In [15]:
min_mean_volume_of_raw_materials = (FIELD_DEVELOP_BUDGET / PROFIT_PER_BARREL)

In [16]:
min_mean_volume_of_raw_materials # минимальный средний объем сырья для региона (тыс. баррелей).

2222.222222222222

In [17]:
min_materials_fields = min_mean_volume_of_raw_materials / max_number_of_deposits

In [18]:
min_materials_fields # минимальный средний объем сырья для месторождения в регионе ( тыс.баррелей)

11.11111111111111

In [19]:
def revenue(target_subsample, probs_subsample, count): # функция расчёта прибыли 
# по набору отобранных месторождений и предсказаний модели.
    pred = probs_subsample.sort_values(ascending=False)
    selected = target_subsample[pred.index][:count]
    revenue = PROFIT_PER_BARREL*selected.sum() - COST_OF_ONE_WELL*count
    return revenue

# 4. Расчёт прибыли и рисков 

In [20]:
state = np.random.RandomState(12345) # случайное распределение

In [21]:
def confidence_interval_and_mean(target_valid, predicted_valid): 
    # бутстреп для расчета доверительного интервала, средней выручки и риска убыточности
    values = []
    for i in range(1000):
        target_valid = pd.Series(target_valid).reset_index(drop=True)
        predicted_valid = pd.Series(predicted_valid).reset_index(drop=True)
        target_subsample = target_valid.sample(n = SEL_BY_REGION, random_state = state, replace = True)
        probs_subsample = predicted_valid[target_subsample.index]
        values.append(revenue(target_subsample, probs_subsample, max_number_of_deposits))
    values = pd.Series(values)
    
    confidence_interval = st.t.interval(0.95, len(values)-1, values.mean(), values.sem())
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    mean_revenue = values.mean()
    risk = stats.percentileofscore(values, 0)
    print('Риск убыточности %: ', risk)
    print('95%-ый доверительный интервал: ', confidence_interval)
    print('2.5% - квантиль: ', lower)
    print('97.5% - квантиль: ', upper)
    print('Средняя выручка: ', mean_revenue)
    
    return confidence_interval, mean_revenue, lower, upper, risk

#### Первый регион.

In [22]:
confidence_interval_0, mean_revenue_0, lower_0, upper_0, risk_0 = confidence_interval_and_mean(target_valid_0, predicted_valid_0)

Риск убыточности %:  0.0
95%-ый доверительный интервал:  (94087322070.68701, 94431448467.52487)
2.5% - квантиль:  88979099051.62064
97.5% - квантиль:  99479763533.58368
Средняя выручка:  94259385269.10594


#### Второй регион.

In [23]:
confidence_interval_1, mean_revenue_1, lower_1, upper_1, risk_1 = confidence_interval_and_mean(target_valid_1, predicted_valid_1)

Риск убыточности %:  0.0
95%-ый доверительный интервал:  (95052498815.76622, 95312691058.18028)
2.5% - квантиль:  91281232314.33087
97.5% - квантиль:  99536129820.66907
Средняя выручка:  95182594936.97325


#### Третий регион.

In [24]:
confidence_interval_2, mean_revenue_2, lower_2, upper_2, risk_2 = confidence_interval_and_mean(target_valid_2, predicted_valid_2)

Риск убыточности %:  0.0
95%-ый доверительный интервал:  (94025287036.50362, 94378593070.3774)
2.5% - квантиль:  88841473908.39989
97.5% - квантиль:  99896299398.44576
Средняя выручка:  94201940053.4405


Вывод: наиболее прибыльный для начала разработок второй регион с максимальной средней выручкой и минимальным доверительным интервалом. Так же все три региона имеют риск убыточности меньше 2.5%